# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# imports

import os
from dotenv import load_dotenv
import gradio as gr
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama
import requests
from bs4 import BeautifulSoup


In [2]:
# Constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key and api_key.startswith("sk-proj-") and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API key looks good so far


In [4]:
# OpenAI instance
openai = OpenAI()

In [5]:
# Tutor system prompt
tutor_system_prompt = """Role and Purpose: You are a tutor for the "Large Language Model Engineering" 
course by Ed Donner. Your role is to provide structured, clear, and patient explanations, helping 
learners understand concepts and apply them effectively.

Start with context, progressing from basic overviews to detailed insights.
Use concise language, real-world examples, and analogies to simplify complex ideas.
Summarize key points, invite further questions and suggest further resources for exploration.

Tone:
Maintain professionalism, avoid assumptions about learners' backgrounds, and focus strictly
on course-relevant content. Use examples and citations aligned with the course material.
"""

In [6]:
# Function for OpenAI GPT response (streaming)
def stream_gpt(message):
    tutor_messages = [
        {"role": "system", "content": tutor_system_prompt},
        {"role": "user", "content": message}
    ]
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=tutor_messages,
        stream=True,
    )
    explanation = ""
    for chunk in stream:
        explanation += chunk.choices[0].delta.content or ""
        yield explanation.replace("```", "").replace("markdown", "")

In [7]:
# Function for Llama response
def explain_with_llama(message):
    tutor_messages = [
        {"role": "system", "content": tutor_system_prompt},
        {"role": "user", "content": message}
    ]
    response = ollama.chat(model=MODEL_LLAMA, messages=tutor_messages)
    return response['message']['content']

In [8]:
# Bonus: Tool for fetching webpage content
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [9]:
# Unified streaming function for Gradio
def stream_explanation(question, model):
    if model == "GPT":
        yield from stream_gpt(question)
    elif model == "Llama":
        explanation = explain_with_llama(question)
        yield explanation
    else:
        raise ValueError("Unknown model")

In [14]:
# Gradio Interface with Markdown rendering for GPT
with gr.Blocks() as ui:
    with gr.Row():
        question_input = gr.Textbox(label="Enter your question:", placeholder="Type your technical question here")
        model_selector = gr.Dropdown(["GPT", "Llama"], label="Select model", value="GPT")
    with gr.Row():
        output_area = gr.Markdown(label="Response:")
    with gr.Row():
        clear_button = gr.Button("Clear")

    def handle_question(question, model):
        if model == "GPT":
            for chunk in stream_gpt(question):  # Stream GPT response
                yield chunk  # Only yield the new chunk
        elif model == "Llama":
            yield explain_with_llama(question)  # Provide Llama response
        else:
            yield "Unknown model selected."

    question_input.submit(handle_question, inputs=[question_input, model_selector], outputs=output_area)
    clear_button.click(lambda: "", None, output_area)

ui.launch()



* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.
